In [ ]:
import os

import pickle

import StaticThreshold as sth

from sklearn.metrics import confusion_matrix

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# Load in Data

In [ ]:
# Load event data generated by Static.ipynb
directory = 'INSERT DIRECTORY'
with open(os.path.join(directory, 'INSERT FILENAME'), 'rb') as f:
    evt_data = pickle.load(f)

In [ ]:
# Load Simulation Data
directory = 'INSERT DIRECTORY' # Directory where events generated by Simulation.py are saved.

files = [] # List of simulation filenames

SNRs = [float(x.split('_')[0]) for x in files]

raw_traces, ground_truths = sth.load_SNR_data(directory, files)

# Tidy Data

In [ ]:
raw_df = pd.Series(raw_traces).to_frame()
raw_df.columns = ['traces']
raw_df.index.names = ['SNR']
raw_df.head()

In [ ]:
truth_df = pd.Series(ground_truths).to_frame()
truth_df.columns = ['ground_truths']
truth_df.index.names = ['SNR']
truth_df.head()

In [ ]:
evt_df = evt_data.to_frame()
evt_df.head()

# Generate Predictions
Convert the saved event locations to one hot encodings

In [ ]:
def my_func(row):
    traces = raw_df.loc[row.index[0][0]][0]
    
    preds = []
    for dev in row.iteritems():
        pred = sth.locs_to_preds(dev[1], traces)
        preds.append(pred)
    return preds

evt_df['predictions'] = evt_df.apply(my_func)

# Score Predictions

In [ ]:
# Create new DataFrame to store scores
columns = ['SNR', 'St_Dev', 'trace_idx', 'FPR', 'TPR', 'FNR', 'TNR']
result_df = pd.DataFrame(columns=columns)
result_df.head()

In [ ]:
for snr, std in list(evt_df.index.unique()):
    print('SNR: ' + str(snr) + ', std: ' + str(std))
    preds = evt_df.loc[snr, std]['predictions'] # Get predictions
    truths = truth_df.loc[snr][0] # Get ground truth
    
    assert preds.shape == truths.shape
    for i in range(preds.shape[0]):
        trace_idx = i
        
        # Generate detection scores
        C = confusion_matrix(truths[i], preds[i], normalize='true')
        tpr = C[1, 1]
        fpr = C[0, 1]
        tnr = C[0, 0]
        fnr = C[1, 0]
        
        # Populate DataFrame
        entry = dict(zip( columns, [[snr], [std], [trace_idx], [fpr], [tpr], [fnr], [tnr]]))                 
        entry_df = pd.DataFrame(entry)
        
        result_df = pd.concat([result_df, entry_df])

# Tidy DataFrame
result_df = result_df.set_index(['SNR', 'St_Dev'])        
result_df.head()

# Save Results

In [ ]:
with open('INSERT FILENAME', 'wb') as f:
    pickle.dump(result_df, f)

# Figure
Generate Example plot showing the prevalance of true and false positives as well as true and false negatives

In [ ]:
snrs = [2, 5]
all_data = []
for snr in snrs:
    f = open(os.path.join(directory, '%d_SNR_data.pkl'%snr), 'rb')
    this_data = pickle.load(f)  
    f.close()
    all_data.append(this_data)
    
all_data_dict = dict(zip(snrs, all_data))    

In [ ]:
def find_pred_evts(pred_evts, window_start, window_end):
    first_evt_idx = 0
    last_evt_idx = pred_evts.shape[0]
    
    for i in range(pred_evts.shape[0]):
        evt = pred_evts[i]
        if evt[0] > window_start:
            first_evt_idx = i
            break
    for i in range(pred_evts.shape[0]):
        evt = pred_evts[i]
        if evt[1] > window_end:
            last_evt_idx = i
            break
            
    return first_evt_idx, last_evt_idx

def plot_event(snr, st_dev, trace_idx, evt_idx, 
               all_data_dict, evt_df, ax, pad=1000, color='red'):
    pred_evts = evt_df.loc[snr, st_dev]['event_locations'][trace_idx]
    trace = all_data_dict[snr][0][trace_idx]
    trace = trace * 1000
    
    trace_true_evts = all_data_dict[snr][2][trace_idx]
    start_idx, end_idx = trace_true_evts[evt_idx]
    start_idx -= pad
    end_idx += pad
    
    first_evt_idx, last_evt_idx = find_pred_evts(pred_evts, start_idx, end_idx)
    
    # x = np.linspace(0, 1, len(trace))
    x = np.linspace(0, len(trace[start_idx:end_idx]), len(trace[start_idx:end_idx])) / 1000000 * 1000
    # ax.plot(x[start_idx:end_idx], trace[start_idx:end_idx])
    ax.plot(x, trace[start_idx:end_idx])
    
    for start, end in pred_evts[first_evt_idx : last_evt_idx]:
        # ax.plot(x[start : end], trace[start : end], color=color)
        ax.plot(x[start - start_idx : end - start_idx], trace[start : end], color=color)
    

fig, axs = plt.subplots(2, 2, figsize=(6, 4), dpi=600, sharey=True, sharex=True)
plot_event(5, 1, 0, 1, all_data_dict, evt_df, ax=axs[0, 0])
plot_event(5, 4, 0, 1, all_data_dict, evt_df, ax=axs[1, 0])
plot_event(2, 1, 0, 1, all_data_dict, evt_df, ax=axs[0, 1])
plot_event(2, 4, 0, 1, all_data_dict, evt_df, ax=axs[1, 1])

axs[1, 0].set_xlabel('Time / ms', weight='bold')
axs[1, 1].set_xlabel('Time / ms', weight='bold')

axs[0, 0].set_ylabel('Current / pA', weight='bold')
axs[1, 0].set_ylabel('Current / pA', weight='bold')

axs[0, 0].annotate('(a)', xy=(0.16, 0), xytext=(0.156, 10), weight='bold')
axs[0, 1].annotate('(b)', xy=(0.16, 0), xytext=(0.156, 10), weight='bold')
axs[1, 0].annotate('(c)', xy=(0.16, 0), xytext=(0.156, 10), weight='bold')
axs[1, 1].annotate('(d)', xy=(0.16, 0), xytext=(0.156, 10), weight='bold')

axs[0, 0].tick_params(width=1.5)
for axis in ['top', 'right', 'bottom', 'left']:
    axs[0, 0].spines[axis].set_linewidth(1.5)
axs[0, 1].tick_params(width=1.5)
for axis in ['top', 'right', 'bottom', 'left']:
    axs[0, 1].spines[axis].set_linewidth(1.5)
axs[1, 0].tick_params(width=1.5)
for axis in ['top', 'right', 'bottom', 'left']:
    axs[1, 0].spines[axis].set_linewidth(1.5)
axs[1, 1].tick_params(width=1.5)
for axis in ['top', 'right', 'bottom', 'left']:
    axs[1, 1].spines[axis].set_linewidth(1.5)

fig.tight_layout()
# axs[0, 0].set(xlim=(2.1, 2.25))

fig.savefig("Example Events.png")